In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
#reading in the routes and poi data
routes = pd.read_csv('cleaned_routes_data/cleaned_all_routes_data_long.csv')
poi = pd.read_csv('all_nodes_uncleaned.csv').iloc[0:1000, :] #using smaller sample of uncleaned data for experimentation

In [23]:
#converting routes df into a GeoDataFrame
gdf_routes = gpd.GeoDataFrame(routes, geometry=gpd.points_from_xy(routes.latitude, routes.longitude))
gdf_routes.drop(['Unnamed: 0', 'lat_lgt'], axis=1, inplace=True)
gdf_routes

In [ ]:
#converting poi df into a GeoDataFrame
gdf_poi = gpd.GeoDataFrame(poi, geometry=gpd.points_from_xy(poi.lat, poi.lon))
gdf_poi

GeoDataFrame-methods: 
crosses(other[, align]) Returns a Series of dtype('bool') with value True for each aligned geometry that cross other.
distance(other[, align]) Returns a Series containing the distance to aligned other.
intersection(other[, align]) Returns a GeoSeries of the intersection of points in each aligned geometry with other.
intersects(other[, align]) Returns a Series of dtype('bool') with value True for each aligned geometry that intersects other.
join(other[, on, how, lsuffix, rsuffix, sort]) Join columns of another DataFrame.
mad([axis, skipna, level]) Return the mean absolute deviation of the values over the requested axis.
sjoin(df, *args, **kwargs) Spatial join of two GeoDataFrames. 
symmetric_difference(other[, align]) Returns a GeoSeries of the symmetric difference of points in each aligned geometry with other.

more: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html